# <span style="color:green"> Python Decorators </span>

**Syntax template:**
> ```Python
> import functools
>
> def decorator(func):
>    @functools.wraps(func)
>     def wrapper_decorator(*args, **kwargs):
>         # Do something before
>         value = func(*args, **kwargs)
>         # Do something after
>         return value
>     return wrapper_decorator
>```

## <span style="color:green"> 1. Functions </span>

Before understanding decorators, understanding of how functions work is needed. 

While not a purely functional language, Python supports many of the functional programming concepts, including **functions as first-class objects.**

### <span style="color:green"> 1.1. Functions as a first-class objects </span>

This means that **functions can be passed around and used as arguments**, just like any other object (string, int, float, list, and so on). 

In [1]:
def say_hello(name):
    return f'Hello, {name}'


def greet_bob(func): # this function expects another function as argument
    return func('Bob')


greet_bob(say_hello) # note, say_hello is w/o parentheses, since it is only a referece name

'Hello, Bob'

### <span style="color:green"> 1.2. Inner functions </span>
It is possible to **define functions inside other functions**. Such functions are called **inner functions.**

Note that the **order in which the inner functions are defined does not matter**. Actions happen when the inner functions are executed.

*Example:*

In [2]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()


parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


Furthermore, the **inner functions are not defined until the parent function is called**. Whenever you call parent function, the inner functions are also called. But because of their local scope, they aren’t available outside of the paren function.

### <span style="color:green"> 1.3. Returning functions from functions </span>

*Example:*

In [3]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child # note that you return func name w/o parentheses
    else:
        return second_child

Note that **you return function without parentheses** i.e. this means that **you are returning the reference to the function.** In contrast function with **parentheses refers to the result of evaluating the function**.

In [4]:
parent(1)()

'Hi, I am Emma'

In [5]:
parent(1)

<function __main__.parent.<locals>.first_child()>

In [6]:
parent

<function __main__.parent(num)>

## <span style="color:green"> 2. Simple Decorators </span>

### <span style="color:green"> 2.1. Introductory examples </span>

#### <span style="color:green"> Example 1 </span>

Put simply: **decorators wrap a function, modifying its behavior.**

But it modifies not a function call, but a functon's definition!

Let’s start with an example:

In [7]:
# Decorator definition: 
def my_decorator(func):  
    def wrapper():
        print('smth happened before')
        func()
        print('smth happened after')
    return wrapper
    
# Function to be modified:
def say_whee():  
    print('   Whee!')
    
# Wrap, decoration:
say_whee = my_decorator(say_whee)

In [8]:
say_whee()

smth happened before
   Whee!
smth happened after


In effect when you wrapped and made decoration in the last line, original function **name** `say_whee()` now points to the `wrapper()` inner function:

In [9]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

i.e. **we can see the chain of functions separated by ` . ` when returning to the fucntion name w/o ` () `.**

**Note:** You can name your inner function whatever you want, and a generic name like `wrapper()` is usually okay. Next, we will name the inner function with the same name as the decorator but with a `wrapper_` prefix.

#### <span style="color:green"> Example 2 </span>

Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. **I.e. we can replace decorators!**

The following example will only run the decorated code during the day:

In [4]:
from datetime import datetime

def not_at_night(func):
    def wrapper():
        if 9 <= datetime.now().hour < 23:
            func()
        else:
            pass
    return wrapper

def say_whee():
    print('Whee!')
    
say_whee = not_at_night(say_whee)

### <span style="color:green"> 2.2. "Pie" syntax - synctatctic sugar </span>
Syntax:
>``` Python
> def <decorator_name>():
>    ...
>
> @ <decorator_name>
> def <original_func>():
    ...
>```

*Example:*

In [5]:
def my_decorator(func):  
    def wrapper():
        print('smth happened before')
        func()
        print('smth happened after')
    return wrapper

@my_decorator # BINGO!
def say_whee():
    print('Whee!')

In [6]:
say_whee()

smth happened before
Whee!
smth happened after


### <span style="color:green"> 2.3. Reusing decorators </span>

Decorator is just a regular Python function. All the **usual tools for easy reusability are available**. 

Create a file called `decorators.py` with the following content:

```Python
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice
```

You can now use this new decorator in other files by doing a **regular import:**

In [7]:
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

In [8]:
say_whee()

Whee!
Whee!


### <span style="color:green"> 2.4. Decorating functions with arguments </span>
Say, we need to decorate a function that accepts some arguments.

Try:


In [ ]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f'Hello, {name}')
    
greet('World')

It will raise an error!

The **problem is** that the inner function `wrapper_do_twice()` does not take any arguments, but `name="World"` was passed to it. You could fix this by letting `wrapper_do_twice()` accept one argument, but then it would not work for the `say_whee()` function you created earlier.

**Solution:** is to **use** `*args` **and** `**kwargs` **in the inner wrapper function**.     
Then it will accept an arbitrary number of positional and keyword arguments.  

Rewrite `decorators.py` as follows:


```Python
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice
```

And try again:

In [27]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [28]:
greet('World')

Hello World
Hello World


### <span style="color:green"> 2.5. Returning values from decorated functions </span>
What happens to the return value of decorated functions?

*Try:*

In [11]:
from decorators import do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

return_greeting("Adam")

Creating greeting
Creating greeting


**Decorator ate the return value from the function.**

**Because the** `do_twice_wrapper()` **doesn’t explicitly return a value**, the call `return_greeting("Adam")` **ended up returning** `None`.

**Solution:** you need to **make wrapper function *returning* the return value of the decorated function.** 

Change your `decorators.py` file:

>```Python
>def do_twice(func):
>     def wrapper_do_twice(*args, **kwargs): # MODIFIED - Accepts arguments of the wrapped function
>         func(*args, **kwargs)
>         value =  func(*args, **kwargs)  # REPLACED
>         return value                    # REPLACED
>      # return [value, value] 
>   return wrapper_do_twice
>```

In [13]:
from decorators_3 import do_twice

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'

### <span style="color:green"> 2.6. Who are you, really? </span>

A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability.      
**Introspection** is the **ability of an object to know about its own attributes at runtime.** 

For instance, a *function knows its own name and documentation:*


In [14]:
print

<function print>

In [16]:
print.__name__

'print'

In [18]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [22]:
print(...)

Ellipsis


Introspection works for functions you define yourself as well:

In [29]:
greet

<function decorators.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

In [31]:
greet.__name__

'wrapper_do_twice'

In [34]:
help(greet)

Help on function wrapper_do_twice in module decorators:

wrapper_do_twice(*args, **kwargs)



**After being decorated, the origial function looses its identity and uses that of the wrapper!** 

**Solution:**  decorators should use the ` @functools.wraps` decorator, which will **preserve information about the original function.**    

Update `decorators.py` again:

>```Python
>import functools  # ADDED
>
>def do_twice(func):
>    @functools.wraps(func)  # ADDED
>     def wrapper_do_twice(*args, **kwargs):
>         func(*args, **kwargs)
>         return func(*args, **kwargs)
>     return wrapper_do_twice
>```

In [36]:
from decorators_4 import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [38]:
greet('Paul')

Hello Paul
Hello Paul


In [40]:
greet

<function __main__.greet(name)>

In [42]:
greet.__name__

'greet'

In [45]:
help(greet)

Help on function greet in module __main__:

greet(name)



**Technical detail:** The ` @functools.wraps` decorator uses the function `functools.update_wrapper()` to update special attributes like `__name__` and `__doc__` that are used in the introspection.

## <span style="color:green"> 3. Real-world use-cases </span>
### <span style="color:green"> 3.1. Timing functions </span>
Create a ` @timer` decorator: measure the **time a function takes to execute** and print the duration to the console. 

*Code:*


In [73]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    # 1
        
        value = func(*args, **kwargs)
        
        end_time = time.perf_counter()      # 2
        run_time = end_time - start_time    # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} s")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
    return "Done!"

In [74]:
waste_some_time(100)

Finished 'waste_some_time' in 0.4642 s


'Done!'

We uses the `time.perf_counter()` function, which does a good job of measuring time intervals.

**Note:** The ` @timer` decorator is great if you just want to get an idea about the runtime of your functions.      
**If you want to do more precise measurements of code, you should instead consider the** `timeit` **module in the standard library.** It temporarily disables garbage collection and runs multiple trials to strip out noise from quick function calls.

### <span style="color:green"> 3.2. Debugging code </span>
` @debug` decorator will **print the arguments a function is called with as well as its return value** every time the function is called:

In [78]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        
        value = func(*args, **kwargs)
        
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

In [79]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [82]:
make_greeting('Andy', '25')

Calling make_greeting('Andy', '25')
'make_greeting' returned 'Whoa Andy! 25 already, you are growing up!'


'Whoa Andy! 25 already, you are growing up!'

This example might not seem immediately useful since the @debug decorator just repeats what you just wrote. **It’s more powerful when applied to small convenience functions that you don’t call directly yourself.**

The following example calculates an approximation to the mathematical constant $e$ based on factorial calculation using a std. lib. function:

In [90]:
import math
from debug import debug

# Apply a decorator to a standard library function !!!
math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e()

Calling factorial(0)
Calling factorial(0)
'factorial' returned 1
'factorial' returned 1
Calling factorial(1)
Calling factorial(1)
'factorial' returned 1
'factorial' returned 1
Calling factorial(2)
Calling factorial(2)
'factorial' returned 2
'factorial' returned 2
Calling factorial(3)
Calling factorial(3)
'factorial' returned 6
'factorial' returned 6
Calling factorial(4)
Calling factorial(4)
'factorial' returned 24
'factorial' returned 24
Calling factorial(5)
Calling factorial(5)
'factorial' returned 120
'factorial' returned 120
Calling factorial(6)
Calling factorial(6)
'factorial' returned 720
'factorial' returned 720
Calling factorial(7)
Calling factorial(7)
'factorial' returned 5040
'factorial' returned 5040
Calling factorial(8)
Calling factorial(8)
'factorial' returned 40320
'factorial' returned 40320
Calling factorial(9)
Calling factorial(9)
'factorial' returned 362880
'factorial' returned 362880
Calling factorial(10)
Calling factorial(10)
'factorial' returned 3628800
'factorial' r

2.7182818284590455

### <span style="color:green"> 3.3. Registering plugins </span>
**Decorators** don’t have to wrap the function they’re decorating. They **can also simply register that a function exists and return it unwrapped.**

This can be used, for instance, to create a light-weight plug-in architecture:

In [93]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

The ` @register` decorator simply stores a reference to the decorated function in the **global** `PLUGINS` dict. You do not have to write an inner function or use ` @functools.wraps` in this example because you are returning the original function unmodified.

The `randomly_greet()` function randomly chooses one of the registered functions to use.    Note that the `PLUGINS` **dictionary already contains references to each function object that is registered** as a plugin:

In [94]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [96]:
randomly_greet('Anna')

Using 'say_hello'


'Hello Anna'

The main benefit of this simple plugin architecture is that you do not need to maintain a list of which plugins exist. That list is created when the plugins register themselves. This makes it trivial to add a new plugin: just define the function and decorate it with ` @register`.